Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
batch_size = 128
hidden = 1024
beta = 0.1

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden]))
  biases = tf.Variable(tf.zeros([hidden]))
  weights1 = tf.Variable(
    tf.truncated_normal([hidden, num_labels]))
  biases1 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  relus = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)
  logits = tf.matmul(relus, weights1) + biases1
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # L2 regularization for the fully connected parameters.
  regularizers = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases) +
                  tf.nn.l2_loss(weights1) + tf.nn.l2_loss(biases1))
  # Add the regularization term to the loss.
  loss += 5e-4 * regularizers

  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
        
    
 
  train_prediction = tf.nn.softmax(logits)
  h1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases)
  valid_prediction = tf.nn.softmax(
    tf.matmul(h1_valid, weights1) + biases1)
  h1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(h1_test, weights1) + biases1)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 470.285400
Minibatch accuracy: 10.2%
Validation accuracy: 38.9%
Minibatch loss at step 500: 139.959244
Minibatch accuracy: 75.8%
Validation accuracy: 81.1%
Minibatch loss at step 1000: 99.830605
Minibatch accuracy: 86.7%
Validation accuracy: 81.3%
Minibatch loss at step 1500: 77.241180
Minibatch accuracy: 82.0%
Validation accuracy: 81.3%
Minibatch loss at step 2000: 58.019566
Minibatch accuracy: 78.9%
Validation accuracy: 82.1%
Minibatch loss at step 2500: 45.049442
Minibatch accuracy: 82.0%
Validation accuracy: 83.6%
Minibatch loss at step 3000: 34.574936
Minibatch accuracy: 82.8%
Validation accuracy: 83.9%
Test accuracy: 90.1%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [7]:
batch_size = 128
hidden = 1024
SEED = 66478  # Set to None for random seed.

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden]))
  biases = tf.Variable(tf.zeros([hidden]))
  weights1 = tf.Variable(
    tf.truncated_normal([hidden, num_labels]))
  biases1 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  relus = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)
    # Add a 50% dropout during training only. Dropout also scales
    # activations such that no rescaling is needed at evaluation time.
  relus = tf.nn.dropout(relus, 0.7, seed=SEED)
  logits = tf.matmul(relus, weights1) + biases1
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # L2 regularization for the fully connected parameters.
  regularizers = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases) +
                  tf.nn.l2_loss(weights1) + tf.nn.l2_loss(biases1))
  # Add the regularization term to the loss.
  loss += 5e-4 * regularizers

  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
        
    
 
  train_prediction = tf.nn.softmax(logits)
  h1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases)
  valid_prediction = tf.nn.softmax(
    tf.matmul(h1_valid, weights1) + biases1)
  h1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(h1_test, weights1) + biases1)

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 578.411316
Minibatch accuracy: 14.8%
Validation accuracy: 30.1%
Minibatch loss at step 500: 140.694000
Minibatch accuracy: 69.5%
Validation accuracy: 80.8%
Minibatch loss at step 1000: 99.323792
Minibatch accuracy: 82.8%
Validation accuracy: 81.0%
Minibatch loss at step 1500: 77.527603
Minibatch accuracy: 72.7%
Validation accuracy: 81.6%
Minibatch loss at step 2000: 57.597900
Minibatch accuracy: 73.4%
Validation accuracy: 82.0%
Minibatch loss at step 2500: 45.114819
Minibatch accuracy: 78.1%
Validation accuracy: 83.0%
Minibatch loss at step 3000: 34.289829
Minibatch accuracy: 87.5%
Validation accuracy: 83.6%
Test accuracy: 90.6%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [26]:
batch_size = 128
hidden = 1024
hidden2 = 300
SEED = 66478  # Set to None for random seed.
beta1=0.01
beta2=0.01
beta3=0.0001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden]))
  biases = tf.Variable(tf.zeros([hidden]))
  weights1 = tf.Variable(
    tf.truncated_normal([hidden, hidden]))
  biases1 = tf.Variable(tf.zeros([hidden]))
  weights2 = tf.Variable(
    tf.truncated_normal([hidden, hidden2]))
  biases2 = tf.Variable(tf.zeros([hidden2]))
  weights3 = tf.Variable(tf.truncated_normal([hidden2, num_labels]))
  biases3 = tf.Variable(tf.zeros([num_labels]))
  
  
    
  relus = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)
  relus2 = tf.nn.relu(tf.matmul(relus, weights1) + biases1)
  relus3 = tf.nn.relu(tf.matmul(relus2, weights2) + biases2)
    # Add a 50% dropout during training only. Dropout also scales
    # activations such that no rescaling is needed at evaluation time.
  #relus = tf.nn.dropout(relus, 0.7, seed=SEED)
  #relus2 = tf.nn.dropout(relus2, 0.7, seed=SEED)

  logits = tf.matmul(relus3, weights3) + biases3
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # L2 regularization for the fully connected parameters.
  regularizers = (beta1*tf.nn.l2_loss(weights) +
                  beta2*tf.nn.l2_loss(weights1) +
                 beta3*tf.nn.l2_loss(weights2) +
                 beta3*tf.nn.l2_loss(weights3))
  # Add the regularization term to the loss.
  #loss += 5e-4 * regularizers
  loss += regularizers

  
  # Optimizer.
  # Training computation.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.04, global_step, 1000, 0.90, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
  
  # Predictions for the training, validation, and test data.
        
    
 
  train_prediction = tf.nn.softmax(logits)
  h1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases)
  h2_valid = tf.nn.relu(tf.matmul(h1_valid, weights1) + biases1)
  h3_valid = tf.nn.relu(tf.matmul(h2_valid, weights2) + biases2)
  valid_prediction = tf.nn.softmax(
    tf.matmul(h3_valid, weights3) + biases3)
  h1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases)
  h2_test = tf.nn.relu(tf.matmul(h1_test, weights1) + biases1)  
  h3_test = tf.nn.relu(tf.matmul(h2_test, weights2) + biases2)  
  test_prediction = tf.nn.softmax(tf.matmul(h3_test, weights3) + biases3)


In [27]:
num_steps = 9001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 91513.687500
Minibatch accuracy: 5.5%
Validation accuracy: 10.0%
Minibatch loss at step 500: nan
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
Minibatch loss at step 1000: nan
Minibatch accuracy: 8.6%
Validation accuracy: 10.0%


KeyboardInterrupt: 